In [1]:
import requests 
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken

/Users/isaachurwitz/opt/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!which python

/Users/isaachurwitz/opt/anaconda3/envs/llm/bin/python


In [5]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
es_client = Elasticsearch('http://localhost:9200') 

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-hw"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-hw'})

In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████| 948/948 [00:03<00:00, 289.46it/s]


Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [11]:
query = "How do I execute a command in a running docker container?"

In [12]:
def elastic_search(query, size=5):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                }
                    }
                }
            }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append((hit['_score'], hit['_source']))
    
    return result_docs

In [13]:
elastic_search(query, 1)

[(84.050095,
  {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'})]

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [14]:
def elastic_search(query, size=5, course=None):
    # Base search query
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    # Add the filter if the course parameter is provided
    if course:
        search_query['query']['bool']['filter'] = {
            "term": {
                "course": course
            }
        }
    
    # Execute the search query
    response = es_client.search(index=index_name, body=search_query)
    
    # Extract the source documents from the response
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    #     result_docs.append((hit['_score'], hit['_source']))
    
    
    return result_docs


In [15]:
res = elastic_search(query, size=3, course='machine-learning-zoomcamp')

for r in res:
    print(r['question'])

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


answer: How do I copy files from a different folder into docker container’s working directory?

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [16]:
def build_context(search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    context = ""

    for r in search_results:
        context += context_template.format(question=r['question'], text=r['text']) + "\n\n"
    return context.strip()

In [17]:
context = build_context(res)

In [18]:
def build_prompt(query, context):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [19]:
prompt = build_prompt(query, context)

In [20]:
len(prompt)

1463

closest answer is 1462

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```


In [21]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [24]:
tokens = encoding.encode(prompt)

In [31]:
len(tokens)

323

The closest answer is 322

In [28]:
orig = encoding.decode_single_token_bytes(tokens[1])

In [30]:
for t in tokens[:20]:
    print(encoding.decode_single_token_bytes(t))

b"You're"
b' a'
b' course'
b' teaching'
b' assistant'
b'.'
b' Answer'
b' the'
b' QUESTION'
b' based'
b' on'
b' the'
b' CONT'
b'EXT'
b' from'
b' the'
b' FAQ'
b' database'
b'.\n'
b'Use'


## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

In [32]:
from openai import OpenAI

In [33]:
client = OpenAI()

In [34]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [35]:
response = llm(prompt)
print(response)

To execute a command in a running Docker container, you can use the `docker exec` command. First, identify the container ID of the running container using the `docker ps` command. Then, use `docker exec` with the appropriate flags. For example, to start an interactive bash session, you would use:

1. List all running containers to find the container ID:
   ```sh
   docker ps
   ```

2. Execute the command in the specific container:
   ```sh
   docker exec -it <container-id> bash
   ```
Replace `<container-id>` with the actual ID of the container you want to execute the command in.


In [36]:
query = 'how do I run kafka?'

def rag(query):
    search_results = elastic_search(query)
    context = build_context(search_results)
    prompt = build_prompt(query, context)
    answer = llm(prompt)
    return answer

In [38]:
query = "How do I execute a command in a running docker container?"
print(rag(query))

To execute a command in a running Docker container, follow these steps:

1. List all running Docker containers to find the container ID:
   ```sh
   docker ps
   ```
2. Execute a command in the specific container using the following command:
   ```sh
   docker exec -it <container-id> <command>
   ```

For example, to start a bash session within a running container, you would use:
```sh
docker exec -it <container-id> bash
```


## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [ ]:
(150 * 1000) + (250 * 1000) 

In [42]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT: 
question: How do I debug a docker container?
answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

question: How do I copy files from my local machine to docker container?
answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_o

In [43]:
len(prompt)

1499